<a href="https://colab.research.google.com/github/yblee110/jax-flax-book/blob/main/ch04_5_DistilGPT2_%EB%AF%B8%EC%84%B8%EC%A1%B0%EC%A0%95%ED%95%99%EC%8A%B5_%EC%A7%84%ED%96%89%ED%95%98%EA%B8%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 마지막 예제 수정 관련된 내용들

마지막 예제의 경우 여러가지 모델들을 실험했을 때 안정적이면서 많은 리소스를 활용하지 않는 선에서 할만한 모델과 데이터셋으로 변경했습니다.

## 모델 : skt/kogpt-2-base-v2
SKT에서 나온 2021년에 나온 모델로 한국어로 이루어져있는 GPT-2 기반 오픈소스 모델입니다.

## 데이터셋 : HAE-RAE의 KOREAN WEBTEXT
HAE-RAE는 양질의 한국어 데이터셋 벤치마크를 구축하고 연구하는 오픈소스 단체입니다. cc100뿐만 아니라 책에 나와있는 oscar데이터셋 CulturaY와 같은 다양한 텍스트 데이터셋을 고품질로 정재한 데이터셋입니다.

In [ ]:
%%capture
!pip install datasets

In [ ]:
import jax.tools.colab_tpu
jax.config.update('jax_xla_backend', 'tpu')
jax.local_devices()

[TpuDevice(id=0, process_index=0, coords=(0,0,0), core_on_chip=0),
 TpuDevice(id=1, process_index=0, coords=(0,0,0), core_on_chip=1),
 TpuDevice(id=2, process_index=0, coords=(1,0,0), core_on_chip=0),
 TpuDevice(id=3, process_index=0, coords=(1,0,0), core_on_chip=1),
 TpuDevice(id=4, process_index=0, coords=(0,1,0), core_on_chip=0),
 TpuDevice(id=5, process_index=0, coords=(0,1,0), core_on_chip=1),
 TpuDevice(id=6, process_index=0, coords=(1,1,0), core_on_chip=0),
 TpuDevice(id=7, process_index=0, coords=(1,1,0), core_on_chip=1)]

In [ ]:
language = "ko"
model_config = "skt/kogpt2-base-v2"
training_seed = 42

In [ ]:
from pathlib import Path


model_dir = model_config + f"-pretrained-{language}"
Path(model_dir).mkdir(parents=True, exist_ok=True)

In [ ]:
from transformers import AutoConfig


config = AutoConfig.from_pretrained(model_config)
config.save_pretrained(f"{model_dir}")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
from datasets import load_dataset
from tokenizers import trainers, Tokenizer, normalizers, ByteLevelBPETokenizer

In [ ]:
#raw_dataset = load_dataset("wikimedia/wikipedia", f"20231101.{language}")
raw_dataset = load_dataset("HAERAE-HUB/KOREAN-WEBTEXT")

Resolving data files:   0%|          | 0/18 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/18 [00:00<?, ?it/s]

Loading dataset shards:   0%|          | 0/17 [00:00<?, ?it/s]

In [ ]:
raw_dataset['train']['text'][0]

'사이트의 판매량에 기반하여 판매량 추이를 반영한 인터파크 도서에서의 독립적인 판매 지수입니다. 현재 가장 잘 팔리는 상품에 가중치를 두었기 때문에 실제 누적 판매량과는 다소 차이가 있을 수 있습니다. 판매량 외에도 다양한 가중치로 구성되어 최근의 이슈도서 확인시 유용할 수 있습니다. 해당 지수는 매일 갱신됩니다. 파일용량 및 다운로드기간 등의 상세 정보는 구매 여부, 대여일에 따라 다를 수 있으므로, 상세페이지에서 확인하세요. 2004년 부커상을 수상한 『아름다움의 선』으로 국내 독자들에게 알려진 작가 앨런 홀링허스트의 “역사적인 데뷔작” 『수영장 도서관』이 ㈜창비에서 출간됐다. 에이즈의 유행과 맞물려 동성애에 대한 사회적 인식이 극도로 악화되었던 대처 수상 집권 말기인 1988년에 출간된 이 소설은, 영국에서 처음으로 남성 동성애자들의 적나라한 성애와 생활을 주류 문학계 안으로 끌어오며 일대 센세이션을 낳았고, 영국과 미국에서 베스트셀러에 오르며 엄청난 반향을 불러일으켰다. 이전까지 B급 하위문화의 한 장르로 취급받던 퀴어소설이 서머싯몸상, 스톤월 도서상, E.M.포스터상 등 굴지의 문학상을 휩쓴 이 작품의 성공에 힘입어 진지한 문학작품으로서 논의되기 시작한 것이다. 이러한 상황에 비추어 『뉴욕 타임스』에서는 앨런 홀링허스트를 가리켜 ‘소설의 게이 해방자’라고 표현했다. 2020년 부커상 수상자인 더글러스 스튜어트는 『수영장 도서관』을 자신의 ‘인생 책’ 중 한권으로 꼽기도 했다. 이 소설에서 최상류층으로 아무 거리낄 것 없이 분방한 생활을 즐기던 젊은 귀족 윌리엄 벡위스가 우연한 만남을 통해 지난 시대 자기 사회의 민낯을 발견해가는 과정은 곧 더없이 탄탄하게 여겨온 자신의 발밑을 허무는 과정이기도 하다. 한편의 추리극처럼 뜻밖의 놀라움을 선사하며 펼쳐지는 중심 이야기를 축으로 격렬한 로맨스와 쓰라린 상실이 섬세하고 예리한 문장, 비틀린 유머와 함께 겹쳐지며 타올랐다 스러지는 청춘의 빛과 그늘을 그려낸다. 제국주의를 지나 신자유주의에 다다르기까지 영국이 나

#### for custom tokenizer

```
def batch_iterator(batch_size=1000):
   for i in range(0, len(raw_dataset), batch_size):
       yield raw_dataset["train"][i: i + batch_size]["text"]


tokenizer = ByteLevelBPETokenizer()


tokenizer.train_from_iterator(batch_iterator(), vocab_size=config.vocab_size, min_frequency=2, special_tokens=[
   "<s>",
   "<pad>",
   "</s>",
   "<unk>",
   "<mask>",
])


tokenizer.save(f"{model_dir}/tokenizer.json")
```

In [ ]:
max_seq_length = 512
raw_dataset["train"] = load_dataset("HAERAE-HUB/KOREAN-WEBTEXT", split="train[5%:]")
raw_dataset["validation"]= load_dataset("HAERAE-HUB/KOREAN-WEBTEXT", split="train[:5%]")


# 전체 데이터셋을 사용하기 위해서는 아래 두 라인은 삭제해도 무방합니다.
raw_dataset["train"] = raw_dataset["train"].select(range(10000))
raw_dataset["validation"] = raw_dataset["validation"].select(range(100))

Resolving data files:   0%|          | 0/18 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/18 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/18 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/18 [00:00<?, ?it/s]

In [ ]:
## (*진우 수정)
## 기존에는 초기학습을 가정하였기 때문에 tokenizer를 새로 만들어주는 코드가 있었음. (2셀 위의 markdown 참조)

from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_config)

def tokenize_function(examples):
   return tokenizer(examples["text"])

tokenized_datasets = raw_dataset.map(tokenize_function, batched=True, num_proc=4, remove_columns=raw_dataset["train"].column_names)
tokenized_datasets


/usr/local/lib/python3.10/dist-packages/multiprocess/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Map (num_proc=4):   0%|          | 0/10000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 10000
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 100
    })
})

In [ ]:
def group_texts(examples):
   concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
   total_length = len(concatenated_examples[list(examples.keys())[0]])
   total_length = (total_length // max_seq_length) * max_seq_length
   result = {
       k: [t[i : i + max_seq_length] for i in range(0, total_length, max_seq_length)]
       for k, t in concatenated_examples.items()
   }
   result["labels"] = result["input_ids"].copy()
   return result

tokenized_datasets = tokenized_datasets.map(group_texts, batched=True, num_proc=4)
tokenized_datasets


Map (num_proc=4):   0%|          | 0/10000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 54831
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 456
    })
})

In [ ]:
import jax
import optax
import flax
import jax.numpy as jnp
import math


from flax.training import train_state
from flax.training.common_utils import get_metrics, onehot, shard


import numpy as np


from tqdm.notebook import tqdm

In [ ]:
per_device_batch_size = 16
num_epochs = 10
learning_rate = 3e-4


total_batch_size = per_device_batch_size * jax.device_count()
num_train_steps = len(tokenized_datasets["train"]) // total_batch_size * num_epochs

In [ ]:
## (*진우 수정)
## 학습된 모델 가져오기 <- 기존에는 밑바닥부터 학습이기 때문에 config로부터 모델의 기본 architecture만 가져옴

from transformers import FlaxAutoModelForCausalLM

model = FlaxAutoModelForCausalLM.from_pretrained(model_config, seed=training_seed, dtype=jnp.dtype("bfloat16"))

In [ ]:
linear_decay_lr_schedule_fn = optax.linear_schedule(init_value=learning_rate, end_value=0, transition_steps=num_train_steps)
adamw = optax.adamw(learning_rate=linear_decay_lr_schedule_fn, b1=0.9, b2=0.98, eps=1e-8, weight_decay=0.01)
state = train_state.TrainState.create(apply_fn=model.__call__, params=model.params, tx=adamw)

In [ ]:
def data_loader(rng, dataset, batch_size, shuffle=False):
   steps_per_epoch = len(dataset) // batch_size


   if shuffle:
       batch_idx = jax.random.permutation(rng, len(dataset))
   else:
       batch_idx = jnp.arange(len(dataset))


   batch_idx = batch_idx[: steps_per_epoch * batch_size]
   batch_idx = batch_idx.reshape((steps_per_epoch, batch_size))


   for idx in batch_idx:
       batch = dataset[idx]
       batch = {k: jnp.array(v) for k, v in batch.items()}


       batch = shard(batch)


       yield batch


In [ ]:
def train_step(state, batch, dropout_rng):
   dropout_rng, new_dropout_rng = jax.random.split(dropout_rng)


   def loss_fn(params):
       labels = batch.pop("labels")
       logits = state.apply_fn(**batch, params=params, dropout_rng=dropout_rng, train=True)[0]


       loss = optax.softmax_cross_entropy(logits[..., :-1, :], onehot(labels[..., 1:], logits.shape[-1])).mean()
       return loss


   grad_fn = jax.value_and_grad(loss_fn)
   loss, grad = grad_fn(state.params)
   grad = jax.lax.pmean(grad, "batch")
   new_state = state.apply_gradients(grads=grad)


   metrics = jax.lax.pmean(
       {"loss": loss, "learning_rate": linear_decay_lr_schedule_fn(state.step)}, axis_name="batch"
   )


   return new_state, metrics, new_dropout_rng


def eval_step(params, batch):
   labels = batch.pop("labels")


   logits = model(**batch, params=params, train=False)[0]


   loss = optax.softmax_cross_entropy(logits[..., :-1, :], onehot(labels[..., 1:], logits.shape[-1])).mean()


   metrics = {"loss": loss, "perplexity": jnp.exp(loss)}
   metrics = jax.lax.pmean(metrics, axis_name="batch")
   return metrics

parallel_train_step = jax.pmap(train_step, "batch")
parallel_eval_step = jax.pmap(eval_step, "batch")


In [ ]:
state = flax.jax_utils.replicate(state)
rng = jax.random.PRNGKey(training_seed)
dropout_rngs = jax.random.split(rng, jax.local_device_count())

In [ ]:

for epoch in tqdm(range(1, num_epochs + 1), desc=f"Epoch ...", position=0, leave=True):
   rng, input_rng = jax.random.split(rng)

   # 학습 단계
   train_loader = data_loader(input_rng, tokenized_datasets["train"], total_batch_size, shuffle=True)
   with tqdm(total=len(tokenized_datasets["train"]) // total_batch_size, desc="Training...", leave=False) as progress_bar_train:
       for model_inputs in train_loader:
           state, train_metric, dropout_rngs = parallel_train_step(state, model_inputs, dropout_rngs)
           progress_bar_train.update(1)

       progress_bar_train.write(
             f"Train... ({epoch}/{num_epochs} | Loss: {round(train_metric['loss'].mean(), 3)}, Learning Rate: {round(train_metric['learning_rate'].mean(), 6)})"
       )

   # 평가 단계
   eval_loader = data_loader(input_rng, tokenized_datasets["validation"], total_batch_size)
   eval_metrics = []

   with tqdm(total=len(tokenized_datasets["validation"]) // total_batch_size, desc="Evaluation...", leave=False) as progress_bar_eval:
       for model_inputs in eval_loader:
           eval_metric = parallel_eval_step(state.params, model_inputs)
           eval_metrics.append(eval_metric)

           progress_bar_eval.update(1)

       eval_metrics = get_metrics(eval_metrics)
       eval_metrics = jax.tree_util.tree_map(jnp.mean, eval_metrics)
       progress_bar_eval.write(
           f"Eval... ({epoch}/{num_epochs} | Loss: {eval_metrics['loss']} | Perplexity: {eval_metrics['perplexity']})"
       )


Epoch ...:   0%|          | 0/10 [00:00<?, ?it/s]

Training...:   0%|          | 0/428 [00:00<?, ?it/s]

Train... (1/10 | Loss: 2.4670000076293945, Learning Rate: 0.0002699999895412475)


Evaluation...:   0%|          | 0/3 [00:00<?, ?it/s]

Eval... (1/10 | Loss: 4.878674507141113 | Perplexity: 157.17214965820312)


...

Eval... (9/10 | Loss: 5.008030414581299 | Perplexity: 181.67422485351562)


Training...:   0%|          | 0/428 [00:00<?, ?it/s]

Train... (10/10 | Loss: 1.8040001392364502, Learning Rate: 0.0)


Evaluation...:   0%|          | 0/3 [00:00<?, ?it/s]

Eval... (10/10 | Loss: 5.053452491760254 | Perplexity: 193.16038513183594)


In [ ]:
## (*진우 수정)
## model param에 학습된 파라미터 반영

model.params = flax.jax_utils.unreplicate(state.params)

In [ ]:
raw_dataset['train']['text'][7]

'<스파이더버스>를 통해서 화려하게 등장했던 ‘스파이더 그웬’은 그야말로 센세이션 그 자체였어요. 책이면 책, 굿즈면 굿즈, 무엇이든 ‘그웬’이라면 불티나게 팔려나가면서 초고속으로 승진해 극장가에 데뷔하기도 했죠(<스파이더맨: 뉴 유니버스>)! <스파이더 그웬>의 첫번째 작가진은 다음 세 사람이에요: 글작가 제이슨 라투어, 그림작가 로비 로드리게즈, 그리고 채색작가 리코. 이 사람들이 대략 3~4년동안 아주 꾸준하게 그웬의 이야기를 냈습니다. 저는 첫번째 그웬 솔타(라투어런)를 처음 읽을 때 꽤나 실망했어서 (그 이유는 후술할게요.) 작가진이 바뀔 때까지 오랫동안 그웬 솔타를 집지 않았어요. 시시콜콜하게 이런 이야기를 하는 이유는, 2015년부터 2018년까지 연재됐었던 라투어런을 굳이 읽지 않아도 바로 맥과이어런으로 정주행을 시작해도 된다는 뜻이에요. 제가 그랬으니까요. 어차피 독자로서 꼭 알아야하는 건 ‘이전 이야기’ 요약을 통해서 다 얘기해주거든요. 쌩뚱맞게 느껴지지 않을 정도로 친절하게 쓰여져있기도 하고요. 우리가 알아야 하는건 1)고등학생때 그웬이 방사능거미에 물려서 스파이더우먼이 되었다. 2)동급생 친구 피터 파커는 리자드가 되어서 비극적인 죽음을 맞았다. 3)그웬은 거미 초능력을 잃었고 4)심비오트 수트를 얻었으며 5)시크릿 아이덴티티를 포기하고 자진투항하여 일년 정도 감옥살이를 하다 나왔다. 이정도 뿐이에요. 정말 쉽죠. 현재 그웬의 나이는 갓 스무살 정도이고, 형을 다 살고 이제 막 출소한 시점에서 이야기가 시작됩니다. 슈퍼히어로 코믹스는 멀티버스 설정을 통해 세계관을 무한하게 확장시켜서 ‘평행우주’라는 이름하에 간편하게 설정변주를 꾀합니다. [누군가 거미에 물리고, 그 누군가는 소중한 사람의 죽음을 겪고, 그로부터 책임감을 배운 뒤에 영웅이 된다.] 그 누군가는 대부분의 경우 미국 뉴욕에 사는 ‘피터 파커’라는 이름의 인물이지만, 종종 피터 파커의 주변인물–해리 오스본, 플래시 톰슨 등–이 되기도 하고, 다른 국가와 도시에 사는 인물이 되기도

In [ ]:
input_text = "<스파이더 그웬>의 첫번째 작가진은 다음 세 사람이에요:"
inputs = tokenizer(input_text, return_tensors="jax")
output = model.generate(inputs["input_ids"],
                        max_length=128,
                        repetition_penalty=2.0,
                        pad_token_id=tokenizer.pad_token_id,
                        eos_token_id=tokenizer.eos_token_id,
                        bos_token_id=tokenizer.bos_token_id,
                        use_cache=True)
generated_text = tokenizer.decode(output[0][0])
print(generated_text)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:51200 for open-end generation.


<스파이더 그웬>의 첫번째 작가진은 다음 세 사람이에요: 톰 크루즈, 조쉬 빌뇌브, 조쉬 빌뇌브, 조쉬 빌뇌브, 조쉬 빌뇌브, 조쉬 빌뇌브. 이 영화는 스파이더 그웬이 스파이더 그웬의 이야기를 다뤘다는 점에서 유사합니다. 이 영화는 스파이더 그웬이 스파이더 그웬의 이야기를 다뤘다는 점에서 유사합니다. 이 영화는 스파이더 그웬이 스파이더 그웬의 이야기를 다뤘다는 점에서 유사합니다. 이 영화는 스파이더 그웬이 스파이더 그웬의 이야기를


### Compare with base model

In [ ]:
base_model = FlaxAutoModelForCausalLM.from_pretrained("skt/kogpt2-base-v2", seed=training_seed, dtype=jnp.dtype("bfloat16"))

input_text = "<스파이더 그웬>의 첫번째 작가진은 다음 세 사람이에요:"
inputs = tokenizer(input_text, return_tensors="jax")
output = base_model.generate(inputs["input_ids"],
                        max_length=128,
                        repetition_penalty=2.0,
                        pad_token_id=tokenizer.pad_token_id,
                        eos_token_id=tokenizer.eos_token_id,
                        bos_token_id=tokenizer.bos_token_id,
                        use_cache=True)
generated_text = tokenizer.decode(output[0][0])
print(generated_text)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:51200 for open-end generation.


<스파이더 그웬>의 첫번째 작가진은 다음 세 사람이에요:
스파이더 그웬의 첫 번째 작가진은 다음 세 사람이에요:
스파이더 그웬의 첫 번째 작가진은 다음 세 사람이에요:
스파이더 그웬의 첫 번째 작가진은 다음 세 사람이에요:
스파이더 그웬의 첫 번째 작가진은 다음 세 사람이에요:
스파이더 그웬의 첫 번째 작가진은 다음 세 사람이에요:
스파이더 그웬의 첫 번째 작가진은 다음 세 사람이에요:
스파이더 그웬의 첫 번째 작가
